In [1]:
from collections import defaultdict
from datetime import datetime
import torch
import pandas as pd

In [2]:
ml_1m_path = '../../../../datasets/ml-1m'

In [3]:
file_path = ml_1m_path + '/movies.dat'
# file_path = Path(file_path)
data_movies = pd.read_csv(file_path, encoding='latin-1', sep='::', engine='python',
                   names=['MovieID', 'Title', 'Genres'], index_col='MovieID')
genres = data_movies['Genres'].str.get_dummies('|')
data_movies= pd.concat([data_movies, genres], axis=1)
# [row, columns], axis = 1 is column
data_movies.drop('Genres', axis=1, inplace=True)
data_movies

,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,Requiem for a Dream (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,Tigerland (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
file_path = ml_1m_path + '/users.dat'
data_users = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], index_col='UserID')

# Convert Gender to binary indicator variable
data_users['Gender'] = (data_users['Gender'] == 'M').astype(int)
occupation_map = {
    0: 'other',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}
# data['Occupation'] = data['Occupation'].map(occupation_map)
data_users

,Gender,Age,Occupation,Zip-code
UserID,,,,
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,02460
5,1,25,20,55455
...,...,...,...,...
6036,0,25,15,32603
6037,0,45,1,76006
6038,0,56,1,14706


In [5]:
file_path = ml_1m_path + '/ratings.dat'

data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'MovieID', 'Rating', 'Timestamp'], index_col='UserID')


In [6]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


## SASRec handling
Only use implicit feedback in the sequence of items

In [7]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [8]:
# use timestamps to determine the sequence order of actions.
actions = data_ratings
actions = actions.groupby('MovieID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID', group_keys=False).apply(lambda  x: x.sort_values('Timestamp'))
# We discard users and items with fewer than 5 related actions.
actions

,MovieID,Rating,Timestamp
UserID,,,
1,3186,4,978300019
1,1721,4,978300055
1,1022,5,978300055
1,1270,5,978300055
1,2340,3,978300103
...,...,...,...
6040,2917,4,997454429
6040,1784,3,997454464
6040,1921,4,997454464


In [9]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
# reorder the userid and itemid (keep the same step with original SASRec code)
for _id, row in actions.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.MovieID in itemmap:
        itemid = itemmap[row.MovieID]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.MovieID] = itemid

In [9]:
usermap.__len__(), itemmap.__len__()

(6040, 3416)

In [10]:
actions['MovieID'] = actions['MovieID'].map(itemmap)
actions.index = actions.index.map(usermap)
# data['Occupation'] = data['Occupation'].map(occupation_map)

In [15]:
actions

,MovieID,Rating,Timestamp
UserID,,,
1,1,4,978300019
1,2,4,978300055
1,3,5,978300055
1,4,5,978300055
1,5,3,978300103
...,...,...,...
6040,1249,4,997454429
6040,88,3,997454464
6040,371,4,997454464


In [14]:
sas_data_T = actions.drop('Rating', axis=1, inplace=False)
sas_data_T

,MovieID,Timestamp
UserID,,
1,1,978300019
1,2,978300055
1,3,978300055
1,4,978300055
1,5,978300103
...,...,...
6040,1249,997454429
6040,88,997454464
6040,371,997454464


In [16]:
with open('ml-1m_time.txt', 'w') as f:
    for _id,_row in sas_data_T.iterrows():
        f.write('%d %d %d\n' % (_id, _row.MovieID, _row.Timestamp))

In [11]:
sas_data = actions.drop('Rating', axis=1, inplace=False).drop('Timestamp', axis=1, inplace=False)
## ..continue
sas_data.loc[1]

,MovieID
UserID,
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9


In [12]:
with open('ml-1m.txt', 'w') as f:
    for _id in set(sas_data.index):
        for movie_id in sas_data.loc[_id].MovieID:
            f.write('%d %d\n' % (_id, movie_id))

In [20]:
def data_partition_window_forSAS(fname, valid_percent, test_percent):
    if valid_percent + test_percent > 0.6:
        print('the percent you select for val/test are too high')
        return None
    valid_start = 1 - valid_percent - test_percent
    test_start = 1 - test_percent
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train_seq = {}
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open('%s.txt' % fname, 'r')
    # read from each line
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)
        # count user and items
    # read from each user
    count = 0
    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:

            # select the whole training seq
            # user_train[user] = User[user][:-2]
            seq_len = len(User[user])
            valid_index = int(seq_len * valid_start)
            test_index = int(seq_len * test_start)
            if valid_index == test_index:
                user_train[user] = User[user]
                user_valid[user] = []
                user_test[user] = []
            else:
                train_seq = User[user][: valid_index]
                valid_seq = User[user][valid_index: test_index]
                test_seq = User[user][test_index:]
                user_train[user] = train_seq
                user_valid[user] = []
                user_valid[user] += valid_seq
                user_test[user] = []
                user_test[user] += test_seq
    return [user_train, user_valid, user_test, usernum, itemnum]

In [24]:
[user_train, user_valid, user_test, usernum, itemnum] = data_partition_window_forSAS('ml-1m', 0.2, 0.2)

In [30]:
len(user_train[1]), len(user_valid[1]), len(user_test[1])

(31, 11, 11)

# Evaluation metrix modification

In [1]:
from collections import defaultdict

def data_partition(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open('%s.txt' % fname, 'r')
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)

    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

In [2]:
dataset = data_partition('ml-1m')

In [3]:
[user_train, user_valid, user_test, usernum, itemnum] = dataset

In [8]:
len(user_train)

6040

In [9]:
len(user_valid)

6040

In [10]:
len(user_test)

6040

In [11]:
usernum

6040

In [12]:
itemnum

3416

In [3]:
import random
import numpy as np
import copy
import torch
import sys

def evaluate_window_valid(model, dataset, args):
    [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
    Recall = 0.0
    P90 = 0.0
    # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
    valid_user = 0.0
    sample_nums = 500
    random_items = random.sample(range(1, itemnum + 1), sample_nums)
    # if usernum > 10000:
    #     # avoid too many training users
    #     # keep at most 10000 users
    #     users = random.sample(range(1, usernum + 1), 10000)
    # else:
    #     # else keep all the users
    #     users = range(1, usernum + 1)
    users = range(1, usernum+1)
    for u in users:
        # make sure the sequence can be validated
        if len(train[u]) < 1 or len(valid[u]) < 1: continue
        seq = np.zeros([args.maxlen], dtype=np.int32)
        idx = args.maxlen - 1
        for i in reversed(train[u]):
            seq[idx] = i
            # fill the sequence from end to beginning
            idx -= 1
            if idx == -1: break
            # select the max len or all of the training data in the sequence
            # limit the length, seq contains the actual training sequence
        rated = set(train[u])
        rated.add(0)
        # all items interacted by the current user
        item_idx = [valid[u][0]]
        # get the index of validated item
        for _ in range(100):
            # negative sampling
            t = np.random.randint(1, itemnum + 1)
            # randomly sample 100 items
            while t in rated: t = np.random.randint(1, itemnum + 1)
            item_idx.append(t)
        predictions = -model.predict(*[np.array(l) for l in [[u], [seq], item_idx]])
        # predicting the recommendation list
        predictions = predictions[0]
        rank = predictions.argsort().argsort()[0].item()
        # the rank of the expected next single item
        valid_user += 1
        if rank < 10:
            Recall += 1
            # P90 coverage
        if valid_user % 100 == 0:
            print('.', end="")
            sys.stdout.flush()
    return Recall / valid_user, P90 / valid_user

In [4]:
import argparse

def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'

def create_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', required=True)
    parser.add_argument('--train_dir', required=True)
    parser.add_argument('--batch_size', default=128, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--maxlen', default=50, type=int)
    parser.add_argument('--hidden_units', default=50, type=int)
    parser.add_argument('--num_blocks', default=2, type=int)
    parser.add_argument('--num_epochs', default=201, type=int)
    parser.add_argument('--num_heads', default=1, type=int)
    parser.add_argument('--dropout_rate', default=0.5, type=float)
    parser.add_argument('--l2_emb', default=0.0, type=float)
    parser.add_argument('--device', default='cpu', type=str)
    parser.add_argument('--inference_only', default=False, type=str2bool)
    parser.add_argument('--state_dict_path', default=None, type=str)
    args = parser.parse_args(args)
    return args

In [5]:
from models.SASRec.model import SASRec
[train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
model_path = '../../processed/ml-1m_repro2/SASRec.epoch=201.lr=0.001.layer=2.head=1.hidden=50.maxlen=200.pth'
# args.device = 'ml-1m'
# args.train_dir = 'test'
# args.state_dict_path = model_path
# args.inference
args = create_args(['--dataset','ml-1m',
                    '--train_dir', 'test',
                    '--device', 'cuda',
                    '--state_dict_path', model_path,
                    '--inference_only', 'true',
                    '--maxlen', '200'])
# print(args.dataset)
model = SASRec(usernum, itemnum, args).to(args.device) # no ReLU activation in original SASRec implementation?
model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))


<All keys matched successfully>

In [6]:
# seq = np.zeros([200], dtype=np.int32)
from collections import Counter

def window_eval(model, dataset, args):
    [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
    Recall = 0.0
    P90 = 0.0
    coverage_list = []
    # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
    valid_user = 0.0
    sample_nums = 100
    random_items = random.sample(range(1, itemnum + 1), sample_nums)
    sample_idx = random_items
    sample_idx_tensor = torch.tensor(sample_idx).to(args.device)
    users = range(1, usernum+1)
    for u in users:
        if len(train[u]) < 1 or len(valid[u]) < 1: continue
        seq = np.zeros([args.maxlen], dtype=np.int32)
        idx = args.maxlen - 1
        for i in reversed(train[u]):
            seq[idx] = i
            # fill the sequence from end to beginning
            idx -= 1
            if idx == -1: break
            # select the max len or all of the training data in the sequence
            # limit the length, seq contains the actual training sequence
        # interacted items
        rated = set(train[u])
        rated.add(0)
        # ground truth item
        ground_truth_idx = [valid[u][0]]
        # collect all indexes, which needs to process on
        process_idx = ground_truth_idx + sample_idx
        predictions = -model.predict(*[np.array(l) for l in [[u], [seq], process_idx]])[0]
        # target distance
        target_d = predictions[0]
        # sampled results
        sample_d = predictions[1:]
        # print(len(sample_d))
        bool_tensor = target_d >= sample_d
        count = torch.sum(bool_tensor).item()
        if count < 10:
            Recall += 1
        sorted_indices = torch.argsort(sample_d)
        sorted_sample_idx = sample_idx_tensor[sorted_indices]
        # take the coverage@10 for all users
        coverage_list+=list(sorted_sample_idx[:10])
        valid_user+=1
    p90_list = [i.item() for i in coverage_list]
    p90_dict = Counter(p90_list)
    p90_sort = sorted(p90_dict.items(), key=lambda x: x[1], reverse=True)
    total_rec = 0
    item_count = 0
    for _, num in p90_sort:
        total_rec+= num
        item_count+= 1
        if total_rec>=0.9*10*usernum:
            break
    return Recall/ valid_user, item_count/sample_nums

In [7]:
r_10, p90_10 = window_eval(model, dataset, args)
r_10, p90_10

(0.6837748344370861, 0.39)

In [324]:
r_10

0.7220198675496688

In [325]:
p90_10[0]

tensor(555, device='cuda:0')

In [310]:
# # rank = predictions
# # rank
# [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
# Recall = 0.0
# P90 = 0.0
# coverage_list = []
# # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
# valid_user = 0.0
# sample_nums = 100
# random_items = random.sample(range(1, itemnum + 1), sample_nums)
# sample_idx = random_items
# users = range(1, usernum+1)
# u = 1
# seq = np.zeros([args.maxlen], dtype=np.int32)
# idx = args.maxlen - 1
# for i in reversed(train[u]):
#     seq[idx] = i
#     # fill the sequence from end to beginning
#     idx -= 1
#     if idx == -1: break
#     # select the max len or all of the training data in the sequence
#     # limit the length, seq contains the actual training sequence
# # interacted items
# rated = set(train[u])
# rated.add(0)
# # ground truth item
# ground_truth_idx = [valid[u][0]]
# process_idx = ground_truth_idx + sample_idx
# predictions = -model.predict(*[np.array(l) for l in [[u], [seq], process_idx]])[0]
# # sampled results
# target_d = predictions[0]
# sample_d = predictions[1:]
# # print(len(sample_d))
#
# bool_tensor = target_d >= sample_d
# # sample_idx = torch.tensor(sample_idx).to(args.device)
# # sample_idx
# # # rank the target result
# count = torch.sum(bool_tensor).item()
# if count < 10:
#     Recall += 1
# sorted_indices = torch.argsort(sample_d)
# print(sample_d)
# sample_idx = torch.tensor(sample_idx).to(args.device)
# sorted_sample_idx = sample_idx[sorted_indices]
# sample_d[sorted_indices], target_d
#
# # if count < 10:
# #     Recall += 1
# # sorted_indices = sample_d.argsort()
# # sorted_sample_idx = sample_idx[sorted_indices]
# # # _, sorted_indices = torch.sort(sample_d, axis=0, direction='ASCENDING', return_sorted_indices=True)
# # # bool_tensor
# # coverage_list.append(list(sorted_sample_idx[:10]))
# # valid_user+=1
# # coverage_list

tensor([ 9.3707, 10.0276,  1.7949,  2.2778,  1.6972,  3.1340,  0.9665,  1.7291,
         3.4084,  7.6006,  4.6640,  2.2345,  0.5467,  5.5857,  5.9262,  1.3569,
         5.1768,  7.5260,  4.5001,  1.7821,  3.2290,  2.4288,  3.4584,  1.6327,
         1.1583,  2.1802,  4.6872,  8.1685,  4.5905,  8.3075,  3.3722,  8.2114,
         0.1550,  9.1146,  3.8488,  3.1302,  7.5430,  1.7347,  2.6680, -0.2786,
         0.3388, -1.0337,  4.5619,  4.3231, -2.1127,  2.3673,  0.5496,  2.3267,
        -0.7152,  0.9132, -0.2480,  3.6144,  2.9695, -0.6554,  6.7954,  8.9082,
         3.2031,  4.8945,  1.2898, -1.0845,  5.7791,  4.0360,  8.3336,  8.1273,
         3.6473,  0.6227, -0.3586, -1.8947,  2.3814,  0.1306, -0.0944, 10.6808,
        -1.7638,  5.1357,  9.9835, -1.3254,  3.1047,  1.2045,  8.4175, -1.1432,
        10.1574,  5.1251, -0.9652,  1.3655,  6.3954,  5.4205,  2.2116, -2.6665,
         0.0877,  7.0560,  6.0650,  6.4022,  1.3186,  1.6585,  3.3703,  1.1821,
         4.2021,  5.0295,  0.1776, -1.28

(tensor([-2.6665, -2.1127, -1.8947, -1.7638, -1.3254, -1.2858, -1.1432, -1.0845,
         -1.0337, -0.9652, -0.7152, -0.6554, -0.3586, -0.2786, -0.2480, -0.0944,
          0.0877,  0.1306,  0.1550,  0.1776,  0.3388,  0.5467,  0.5496,  0.6227,
          0.9132,  0.9665,  1.1583,  1.1821,  1.2045,  1.2898,  1.3186,  1.3569,
          1.3655,  1.6327,  1.6585,  1.6972,  1.7291,  1.7347,  1.7821,  1.7949,
          2.1802,  2.2116,  2.2345,  2.2778,  2.3267,  2.3673,  2.3814,  2.4288,
          2.6680,  2.9695,  3.1047,  3.1302,  3.1340,  3.2031,  3.2290,  3.3703,
          3.3722,  3.4084,  3.4584,  3.6144,  3.6473,  3.8488,  4.0360,  4.2021,
          4.3231,  4.5001,  4.5619,  4.5905,  4.6640,  4.6872,  4.8945,  5.0295,
          5.1251,  5.1357,  5.1768,  5.4205,  5.5857,  5.7791,  5.9262,  6.0650,
          6.3954,  6.4022,  6.7954,  7.0560,  7.5260,  7.5430,  7.6006,  8.1273,
          8.1685,  8.2114,  8.3075,  8.3336,  8.4175,  8.9082,  9.1146,  9.3707,
          9.9835, 10.0276, 1